In [ ]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨.csv')
customer = pd.read_csv('해운대 입장객수2.csv')

In [ ]:
weather.head(5)

,날짜,강수_관측값,기온,습도,체감온도,평균풍속,평균기압,평균수온,평균최대파고,평균파주기
0,2012-05-01,10.5,20.4,62.1,21.5,6.5,1009.0,15.2,1.2,4.8
1,2012-05-02,1.0,19.6,70.5,21.4,7.8,1004.0,15.5,2.1,5.7
2,2012-05-03,0.1,20.7,77.5,22.9,4.1,1004.0,16.4,1.9,6.6
3,2012-05-04,0.0,19.9,70.7,21.7,4.1,1006.5,16.4,1.6,7.5
4,2012-05-05,0.0,24.8,47.2,24.7,6.1,1004.9,15.8,1.9,7.8


In [ ]:
customer.head(5)

,방문일,방문객수
0,2012-05-01,NaN
1,2012-05-02,NaN
2,2012-05-03,NaN
3,2012-05-04,NaN
4,2012-05-05,NaN


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
from sklearn.model_selection import train_test_split
import inspect, os


# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수'] = customer['방문객수'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

AttributeError: ignored

In [ ]:
weather.shape

(1618, 10)

In [ ]:
customer.shape

(1465, 2)

In [ ]:
# merge data : 방문객수 예측을 위한 데이터
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수"]]

# weather data : 날씨 예측을 위한 데이터(na는 제거하는 방향으로 일단 진행)
total_data.loc[total_data['평균수온'] != total_data['평균수온'], '평균수온'] = total_data['평균수온'].mean()
total_data.loc[total_data['평균풍속'] != total_data['평균풍속'], '평균풍속'] = total_data['평균풍속'].mean()
total_data.loc[total_data['평균기압'] != total_data['평균기압'], '평균기압'] = total_data['평균기압'].mean()
total_data.loc[total_data['평균최대파고'] != total_data['평균최대파고'], '평균최대파고'] = total_data['평균최대파고'].mean()
total_data.loc[total_data['평균파주기'] != total_data['평균파주기'], '평균파주기'] = total_data['평균파주기'].mean()
total_data = total_data.fillna(0)

In [ ]:
total_data.isnull().sum()

강수_관측값    0
기온        0
습도        0
체감온도      0
평균수온      0
평균풍속      0
평균기압      0
평균최대파고    0
평균파주기     0
방문객수      0
dtype: int64

## 방문객 수 예측

In [ ]:
# 방문객 수 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(total_data.iloc[:, :-1], total_data['방문객수'], test_size=0.2)

In [ ]:
# 방문객 수 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [ ]:
train_x_scaled.shape

(1172, 9)

In [ ]:
test_x_scaled.shape

(293, 9)

In [ ]:
# 방문객 수 예측을 위한  model training
model=Sequential()
model.add(Dense(128,input_dim=9, activation='relu'))
model.add(Dense(256, activation='swish'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='swish'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='swish'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='swish'))
model.add(Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 128)               1280      
                                                                 
 dense_20 (Dense)            (None, 256)               33024     
                                                                 
 dense_21 (Dense)            (None, 512)               131584    
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 dense_23 (Dense)            (None, 128)               32896     
                                                                 
 dense_24 (Dense)            (None, 64)                8256      
                                                                 
 dense_25 (Dense)            (None, 32)               

In [ ]:
# 학습 진행
earlyStopping=EarlyStopping(monitor='val_loss',patience=500,mode='min',verbose=1,restore_best_weights=True) 
model.compile(loss='mse',optimizer='adam')
hist=model.fit(train_x_scaled,train_y,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[earlyStopping])

Epoch 1/3000
8/8 [==============================] - 1s 41ms/step - loss: 23353497600.0000 - val_loss: 19023800320.0000
Epoch 2/3000
8/8 [==============================] - 0s 17ms/step - loss: 23352766464.0000 - val_loss: 19021209600.0000
Epoch 3/3000
8/8 [==============================] - 0s 15ms/step - loss: 23344359424.0000 - val_loss: 18996983808.0000
Epoch 4/3000
8/8 [==============================] - 0s 16ms/step - loss: 23275829248.0000 - val_loss: 18832791552.0000
Epoch 5/3000
8/8 [==============================] - 0s 16ms/step - loss: 22876737536.0000 - val_loss: 17975658496.0000
Epoch 6/3000
8/8 [==============================] - 0s 16ms/step - loss: 21076637696.0000 - val_loss: 15109910528.0000
Epoch 7/3000
8/8 [==============================] - 0s 16ms/step - loss: 17695315968.0000 - val_loss: 15417337856.0000
Epoch 8/3000
8/8 [==============================] - 0s 17ms/step - loss: 17507532800.0000 - val_loss: 13884024832.0000
Epoch 9/3000
8/8 [==============================

In [ ]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 입장객 수: ', pred_y[-1:])

10/10 [==============================] - 0s 3ms/step - loss: 16140721152.0000
loss:  16140721152.0
예상 입장객 수:  [[28800.125]]


## 강수_관측값 예측

In [ ]:
# 사용 데이터
weather.info()

# input data
weather_input = weather[['강수_관측값',"기온", "습도", "체감온도", "평균풍속","평균기압", "평균수온", "평균최대파고", "평균파주기"]]
weather_output = weather[['강수_관측값',"기온"]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1618 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1618 non-null   datetime64[ns]
 1   강수_관측값  1618 non-null   float64       
 2   기온      1618 non-null   float64       
 3   습도      1618 non-null   float64       
 4   체감온도    1618 non-null   float64       
 5   평균풍속    1567 non-null   float64       
 6   평균기압    1557 non-null   float64       
 7   평균수온    1511 non-null   float64       
 8   평균최대파고  1552 non-null   float64       
 9   평균파주기   1551 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 126.5 KB


In [ ]:
# 강수_관측값 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input.values, weather_output.values, test_size=0.2)

In [ ]:
# 강수_관측값 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [ ]:
train_x_scaled.shape

(1294, 9)

In [ ]:
test_x_scaled.shape

(324, 9)

In [ ]:
# 강수_관측값 예측을 위한 data reshape
train_x_scaled = train_x_scaled.reshape(-1, 3, 3)
train_x_scaled = train_x_scaled.astype(float)
test_x_scaled = test_x_scaled.reshape(-1, 3, 3)
test_x_scaled = test_x_scaled.astype(float)

In [ ]:
train_y = train_y.astype(float)
test_y = test_y.astype(float)

In [ ]:
# 강수_관측값 예측을 위한 lstm 학습
model=Sequential()
model.add(LSTM(units=128,input_shape=(3,3)))
model.add(Dense(64, activation='relu'))
model.add(Dense(2))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               67584     
                                                                 
 dense_28 (Dense)            (None, 64)                8256      
                                                                 
 dense_29 (Dense)            (None, 2)                 130       
                                                                 
Total params: 75,970
Trainable params: 75,970
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 학습 진행
earlyStopping = EarlyStopping(monitor='val_loss', patience=30, mode='min', verbose=1,restore_best_weights=True)
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
hist = model.fit(train_x_scaled, train_y, epochs=500, batch_size=32, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

NameError: ignored

In [ ]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 강수_관측값,기온 : ', pred_y[-1:])

11/11 [==============================] - 0s 4ms/step - loss: 176.0946
loss:  176.09461975097656
예상 강수_관측값,기온 :  [[10.123506 27.850018]]


## 기온 예측

In [ ]:
# 사용 데이터
weather.info()

# input data
weather_input = weather[["강수_관측값", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = weather[['기온']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1486 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1486 non-null   datetime64[ns]
 1   강수_관측값  1486 non-null   float64       
 2   기온      1486 non-null   float64       
 3   습도      1486 non-null   float64       
 4   체감온도    1486 non-null   float64       
 5   평균풍속    1486 non-null   float64       
 6   평균기압    1486 non-null   float64       
 7   평균수온    1486 non-null   float64       
 8   평균최대파고  1486 non-null   float64       
 9   평균파주기   1486 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 127.7 KB


In [ ]:
# 기온 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input.values, weather_output.values, test_size=0.2)

In [ ]:
# 기온 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [ ]:
# 기온 예측을 위한 data reshape
train_x_scaled = train_x_scaled.reshape(-1, 8, 1)
test_x_scaled = test_x_scaled.reshape(-1, 8, 1)

In [ ]:
# 기온 예측을 위한 lstm 학습
model=Sequential()
model.add(LSTM(units=128,input_shape=(8,1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_57 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_58 (Dense)             (None, 1)                 65        
Total params: 74,881
Trainable params: 74,881
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 학습 진행
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1,restore_best_weights=True)
model.compile(loss='mse', optimizer='adam')
hist = model.fit(train_x_scaled, train_y, epochs=500, batch_size=16, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

Epoch 1/500
60/60 - 6s - loss: 294.8029 - val_loss: 16.9127
Epoch 2/500
60/60 - 2s - loss: 14.5535 - val_loss: 14.3196
Epoch 3/500
60/60 - 2s - loss: 14.2070 - val_loss: 14.2903
Epoch 4/500
60/60 - 2s - loss: 14.2028 - val_loss: 14.9367
Epoch 5/500
60/60 - 2s - loss: 14.2439 - val_loss: 14.2816
Epoch 6/500
60/60 - 2s - loss: 14.4665 - val_loss: 14.3856
Epoch 7/500
60/60 - 2s - loss: 14.2717 - val_loss: 14.2550
Epoch 8/500
60/60 - 1s - loss: 14.1948 - val_loss: 14.6336
Epoch 9/500
60/60 - 1s - loss: 14.6506 - val_loss: 14.8402
Epoch 10/500
60/60 - 1s - loss: 14.4735 - val_loss: 14.4695
Epoch 11/500
60/60 - 1s - loss: 14.1548 - val_loss: 14.2540
Epoch 12/500
60/60 - 1s - loss: 14.2216 - val_loss: 14.6555
Epoch 13/500
60/60 - 1s - loss: 14.4435 - val_loss: 15.0292
Epoch 14/500
60/60 - 1s - loss: 14.4140 - val_loss: 14.2505
Epoch 15/500
60/60 - 2s - loss: 14.2542 - val_loss: 14.8203
Epoch 16/500
60/60 - 1s - loss: 14.3089 - val_loss: 14.4372
Epoch 17/500
60/60 - 1s - loss: 14.2792 - val_lo

In [ ]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 기온 예측값 : ', pred_y[-1:])

10/10 [==============================] - 0s 4ms/step - loss: 12.3377
loss:  12.337726593017578
예상 기온 예측값 :  [[26.389303]]
